# Interleave headers
## 2022-01-16
- I first grepped the various headers in the text email files from the command line. 
- Each line of each files contains the container file, the line number, and the header, and the remaining line items. 
- Objective is to construct a single file with the headers in a chronological order, and from there construct a list of proper emails. Hopefully taking the threads into account

In [4]:
import numpy as np
import pandas as pd
import os, re
from datetime import datetime
from glob import glob

In [5]:
inputpath="./email_txt/downloaded_from_data_tallahassee/"
files = glob(inputpath + "*.txt")

In [6]:
files_dict = {}
for file in files[0:5]:
    with open(file, "r") as f:
        files_dict[file] = f.read().splitlines(); # remove \n 

In [9]:
for k in files_dict.keys(): print(k)

./email_txt/downloaded_from_data_tallahassee/23-11-PCSgmail2014-2017.txt
./email_txt/downloaded_from_data_tallahassee/15-6-HunterHarp2014-1.txt
./email_txt/downloaded_from_data_tallahassee/17-2-IB2013-1-0.txt
./email_txt/downloaded_from_data_tallahassee/34-1-ScottMaddox1-Redacted.txt
./email_txt/downloaded_from_data_tallahassee/18-5-Inkbridge2013-3-0.txt


In [10]:
for k in files_dict.keys():
    # print(len(files_dict[k]))
    pass

In [11]:
prefixes = ['date', 'sent', 'to', 'cc', 're', 'bcc', 'when', 'subject', 'original', 'from']
folder = "search_results"
# prefixes = ['date']

In [12]:
breakup_date = re.compile(r'([\w-]+?.txt):(\d+?):(.*)')

In [584]:
dicts = {}
for prefix in prefixes:
    inputpath = os.path.join(folder, prefix + "_txt")
    folder_list = []
    dicts[prefix] = folder_list

    with open(inputpath, "r", encoding="utf8") as f:
        print("inputpath: ", inputpath)
        for line in f.readlines():
            filenm, ln, rest = breakup_date.findall(line)[0]
            # grep -n numbers lines from 1, and I want 0-based
            d = {"filenm": filenm, "ln": int(ln)-1, "rest": rest.rstrip()}
            folder_list.append(d)

inputpath:  search_results/date_txt
inputpath:  search_results/sent_txt
inputpath:  search_results/to_txt
inputpath:  search_results/cc_txt
inputpath:  search_results/re_txt
inputpath:  search_results/bcc_txt
inputpath:  search_results/when_txt
inputpath:  search_results/subject_txt
inputpath:  search_results/original_txt
inputpath:  search_results/from_txt


In [585]:
dfs = {}
for k in dicts.keys():
    dfs[k] = pd.DataFrame(dicts[k])
    print(k, dfs[k].shape)

date (15465, 3)
sent (58640, 3)
to (71105, 3)
cc (36448, 3)
re (33718, 3)
bcc (172, 3)
when (647, 3)
subject (74181, 3)
original (1, 3)
from (71557, 3)


In [586]:
dates = dfs['date']
dates.shape

(15465, 3)

In [587]:
dfs['date'].head()

,filenm,ln,rest
0,10-1-Cascade-2013.txt,133,"Date: Thursday, July 11, 2013 1:08 PM"
1,10-1-Cascade-2013.txt,1381,"Date: August 20, 2013, 6:24:26 PM EDT"
2,10-1-Cascade-2013.txt,1722,"Date: Tue, Jun 25, 2013 at 11:45 AM"
3,10-1-Cascade-2013.txt,2106,"Date: Tue, Jun 25, 2013 at 11:45 AM"
4,10-1-Cascade-2013.txt,2127,"Date: Tue, Jun 25, 2013 at 11:45 AM"


In [588]:
v = dates['rest'].values
len(v)

15465

In [1]:
rest = dfs['date'].loc[:, 'rest'].values
rest = [r.strip() for r in rest]
rest = [r[5:].strip() if r[0:5].lower() == 'date:' else r.strip() for r in rest]
for i, r in enumerate(rest):
    rest[i] = re.sub("\(GMT-05:00\)", "", r)
dfs['date']['rest'] = rest
len(rest)

NameError: name 'dfs' is not defined

In [601]:
dfs.keys()

dict_keys(['date', 'sent', 'to', 'cc', 're', 'bcc', 'when', 'subject', 'original', 'from'])

In [602]:
rest = dfs['sent'].loc[:, 'rest'].values
rest = [r.strip() for r in rest]
rest = [r[5:].strip() if r[0:5].lower() == 'sent:' else r.strip() for r in rest]
for i, r in enumerate(rest):
    rr = re.sub("\(GMT-05:00\).*", "", r)
    rr = re.sub("\(UTC-05:00\).*", "", rr)
    rest[i] = rr
dfs['sent']['rest'] = rest
len(rest)

58640

In [603]:
dfs['date'].shape, len(rest)

((15465, 3), 58640)

In [604]:
dfs['date'].head()

,filenm,ln,rest
0,10-1-Cascade-2013.txt,133,"Thursday, July 11, 2013 1:08 PM"
1,10-1-Cascade-2013.txt,1381,"August 20, 2013, 6:24:26 PM EDT"
2,10-1-Cascade-2013.txt,1722,"Tue, Jun 25, 2013 at 11:45 AM"
3,10-1-Cascade-2013.txt,2106,"Tue, Jun 25, 2013 at 11:45 AM"
4,10-1-Cascade-2013.txt,2127,"Tue, Jun 25, 2013 at 11:45 AM"


In [605]:
dfs['sent'].head()

,filenm,ln,rest
0,10-1-Cascade-2013.txt,1,"Sunday, July 14, 2013 2:39 PM"
1,10-1-Cascade-2013.txt,20,"Sunday, July 14, 2013 2:39 PM"
2,10-1-Cascade-2013.txt,39,"Monday, July 15, 2013 8:18 AM"
3,10-1-Cascade-2013.txt,192,"Monday, July 15, 2013 8:35 AM"
4,10-1-Cascade-2013.txt,230,"Monday, July 15, 2013 8:35 AM"


In [606]:
for i in range(49):
    print(i, dfs['date'].loc[i, 'rest'])

0 Thursday, July 11, 2013 1:08 PM
1 August 20, 2013, 6:24:26 PM EDT
2 Tue, Jun 25, 2013 at 11:45 AM
3 Tue, Jun 25, 2013 at 11:45 AM
4 Tue, Jun 25, 2013 at 11:45 AM
5 November 19, 2013 at 7:51:38 PM EST
6 November 19, 2013 at 7:51:38 PM EST
7 November 26, 2013 at 11:06:32 AM EST
8 November 26, 2013 at 11:06:32 AM EST
9 November 26, 2013 at 11:06:32 AM EST
10 December 10, 2013 at 4:32:37 PM EST
11 Tue, Dec 10, 2013 2:06 PM
12 December 10, 2013 at 4:32:37 PM EST
13 Tue, Dec 10, 2013 2:06 PM
14 December 10, 2013 at 4:32:37 PM EST
15 Tue, Dec 10, 2013 2:06 PM
16 December 10, 2013 at 4:32:37 PM EST
17 Tue, Dec 10, 2013 2:06 PM
18 December 10, 2013 at 4:32:37 PM EST
19 Tue, Dec 10, 2013 2:06 PM
20 December 10, 2013 at 4:32:37 PM EST
21 Tue, Dec 10, 2013 2:06 PM
22 December 10, 2013 at 4:32:37 PM EST
23 Tue, Dec 10, 2013 2:06 PM
24 December 10, 2013 at 4:32:37 PM EST
25 Tue, Dec 10, 2013 2:06 PM
26 December 10, 2013 at 4:32:37 PM EST
27 Tue, Dec 10, 2013 2:06 PM
28 December 10, 2013 at 4:32:37

In [607]:
dates1 = pd.to_datetime(dfs['date'].loc[:, 'rest'], errors='coerce');
# lines of data_indexes translated to NaT, so are not valid date-times
NaT_date_indexes = dates1[dates1.isna() == True].index

/Users/erlebach/opt/anaconda3/envs/torch/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/erlebach/opt/anaconda3/envs/torch/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/erlebach/opt/anaconda3/envs/torch/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PMEDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future 

In [608]:
dates1.shape

(15465,)

d = dates.iloc[NaT_date_indexes,:]
for i in range(1000):
    print(d.iloc[i,:])

In [609]:
sent1 = pd.to_datetime(dfs['sent'].loc[:, 'rest'], exact=False, errors='coerce');
# lines of data_indexes translated to NaT, so are not valid date-times
NaT_sent_indexes = dates1[sent1.isna() == True].index
sent_NaT = dfs['sent'].iloc[NaT_sent_indexes]
NaT_sent_indexes.shape

(244,)

In [644]:
# Most errors start with >Sent, which are not original emails, but rather, Re, Fwd, or emails inside threads. 
# So we can remove them. On the other side, 
# sent_NaT.head(50)
count = 0
inputpath="./email_txt/downloaded_from_data_tallahassee/"

rows = []
for row in sent_NaT.iterrows():
    # Only print the lines that do not start with >
    # if sent_NaT.loc[i, 'rest'].value != '>':a
    ln = int(row[1][1])
    fn = row[1][0]
    rest = row[1][2]
    # print("fn: ", fn)
    # print("inputpath: ", inputpath)
    # print("files_dict.keys(): ", files_dict.keys())
    # lines = files_dict[inputpath+fn]
    # print("lines: ", lines[ln : ln+1], "\n")
    # print("ln: ", ln)
    if row[1][2][0] != '>':
        lines = files_dict[inputpath+fn]
        # Took care of zero-based line numbers earlier in this file
        # print("lines: ", lines[ln : ln+2])
        # print(fn, ln, rest)
        # print()
        Lines = " ".join(lines[ln:ln+10])
        # Remove "Sent:" if it occurs at the front
        Lines = re.sub("\b.*?Sent:", "", Lines)
        # Remove leading ">" denoting threads
        Lines = re.sub(">+? Sent:", "", Lines)
        # Remove from To: onward 
        Lines = re.sub("(.*?)(To:.*)", r"\1", Lines)
        Lines = re.sub("\*\*", "", Lines)
        # Fix "...Tuesday August 26:" into "...Tuesdsay August 26, "...
        Lines = re.sub("([a-z] [0-9]*?):", r"\1, ", Lines)
        # Replace ": " by ":" to handle bad time
        Lines = re.sub(": ", ":", Lines)

        # print("row: ", row)
        rows.append(Lines)
        # print("Lines: ", Lines)
        count += 1
        #print("-------------------------")
    #( print(sent_NaT.loc[i, 'rest'])
print("count= ", count)
ddf = pd.DataFrame({'col':rows})
datess = pd.to_datetime(ddf['col'], exact=False, errors='coerce')
print(datess)
print(ddf)
"""
31   Thursday, April 09, 2015 06:04 AM US Mountain...
32   Thursday, April 09, 2015 06:04 AM US Mountain...
Perhaps add a column of time corrections? mostly 0, but +2 hours if US Mountain ...? 
"""


count=  56
0                    NaT
1                    NaT
2                    NaT
3    2016-02-21 13:54:00
4    2016-02-21 13:54:00
5    2016-02-21 13:54:00
6    2016-02-21 13:54:00
7    2017-01-13 16:07:00
8    2017-01-13 15:16:00
9    2017-01-13 11:53:00
10                   NaT
11                   NaT
12   2017-01-10 16:36:00
13   2017-01-10 15:13:00
14   2017-01-10 16:36:00
15   2017-01-10 15:13:00
16   2017-01-13 16:07:00
17   2017-01-13 15:16:00
18   2017-01-13 11:53:00
19                   NaT
20                   NaT
21   2017-01-10 16:36:00
22   2017-01-10 15:13:00
23   2017-01-10 16:36:00
24   2017-01-10 15:13:00
25   2012-12-20 13:26:00
26   2012-12-20 13:16:00
27   2012-12-20 10:54:00
28   2012-12-20 13:26:00
29   2012-12-20 13:16:00
30   2012-12-20 10:54:00
31                   NaT
32                   NaT
33   2014-08-26 15:07:00
34   2012-12-20 14:03:00
35   2012-12-20 15:15:00
36   2012-12-20 14:03:00
37   2012-12-20 14:03:00
38   2012-12-20 13:26:00
39   2012-12-2

'\n31   Thursday, April 09, 2015 06:04 AM US Mountain...\n32   Thursday, April 09, 2015 06:04 AM US Mountain...\nPerhaps add a column of time corrections? mostly 0, but +2 hours if US Mountain ...? \n'